[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/liyangyang515/Spatio-Temporal-Patterns-of-NO2-and-Mobility-Through-the-Variants-of-COVID-19-in-SEA/blob/main/code/NO2_From_GEE_SEA.ipynb)



## Install geemap

In [ ]:
#install geemap first when running in Colab or new environment
#!pip install geemap



## Import Libraries

In [ ]:
import os
import ee
import geemap
import datetime

In [ ]:
# geemap.update_package()

## Import Study Area (South-East Asia) Points

In [ ]:
Map = geemap.Map()
Map

In [ ]:
# Import the points where NO2 values will be pulled from GEE
in_csv = 'https://raw.githubusercontent.com/liyangyang515/NO2-in-South-East-Asia-_GE5219/main/data/Points_locations/SEA_pts.csv'

In [ ]:
pts = geemap.csv_to_pandas(in_csv)
pts

In [ ]:
# convert to ee points
ee_SEA = geemap.pandas_to_ee(pts, latitude='lat', longitude='lon')

In [ ]:
# add points layer to map
Map.addLayer(ee_SEA, {}, 'SEA')

## Create a Loop to Export Daily Means

In [ ]:
# get daily mean from Mar 1, 2020 to Dec 31, 2021
InitialDate= datetime.datetime(2020,3,1)
# loop within the study period
for i in range(0,730,1):
    StartDate = (InitialDate + datetime.timedelta(days=i)).strftime('%Y-%m-%d')
    EndDate = (InitialDate + datetime.timedelta(days=(i+1))).strftime('%Y-%m-%d')
    collection = ee.ImageCollection('COPERNICUS/S5P/NRTI/L3_NO2') \
    .select('tropospheric_NO2_column_number_density') \
    .filterDate(StartDate, EndDate) 
    # Reduce the collection with a mean reducer.
    mean = collection.reduce(ee.Reducer.mean())
    # output daily mean to a csv for each day
    out_csv = r'D:\GitHub\NO2-in-South-East-Asia-_GE5219\data\NO2_SEA_Land\NO2_raw_daily\NO2_' + StartDate + '.csv'
    geemap.extract_values_to_points(ee_SEA,mean,out_csv, scale = 27750)
